# Example 1 - Simple p2p Connection

Example 1 demonstrates how we can setup a simple 2-way p2p OSC communication between Python and Pure Data. First, we define a client and semd 10 "hello" messages to a remote client (PD). Then, we set up a simple server that listens for OSC messages from remote clients.

https://python-osc.readthedocs.io/en/latest/index.html

### Files 
1. Notebook "Example 1 - Simple p2p"
2. Pure Data patch example1.pd from "../pd/example1.pd"

<p align="left">
 <img src="../../fig/example1.jpg" width=500>
</p>

In [ ]:
# setup our connection variables 
# remote client ip and port
clientIp = '127.0.0.1'
clientPort = 8001

# local server ip and port
serverIp = '127.0.0.1'
serverPort = 8000

## Connect with PD client

In [ ]:
from pythonosc import udp_client

# Setup a client connection and send messages to PD
client = udp_client.UDPClient(clientIp, clientPort)
print(f'Starting client on {clientIp}, port {clientPort}.')

In [ ]:
from pythonosc import osc_bundle_builder
from pythonosc import osc_message_builder
import time

# send 10 OSC frames (custom bundles) with a messages to a client
for i in range(10):
    # open a OSC bundle
    bundle = osc_bundle_builder.OscBundleBuilder(
        osc_bundle_builder.IMMEDIATELY)

    # create a message with an OSC address
    msg = osc_message_builder.OscMessageBuilder(address="/pd")

    # add arguments to the message
    msg.add_arg(f'Hello nr.{i} from python client!')

    # build the message and add to current bundle
    bundle.add_content(msg.build())

    # close OSC bundle
    bundle = bundle.build()

    # send the bundle to remote client
    client.send(bundle)

    # One frame every half second
    time.sleep(.5)

print("done sending...")

## Serve an OSC server

In [ ]:
from pythonosc import dispatcher
from pythonosc import osc_server

# Setup different "routes" where we can map different functions to different OSC adressess we receive.
def pyHandler(address, args):
    print(f'{address} {args}')

dispatcher = dispatcher.Dispatcher()
dispatcher.map("/py*", pyHandler)

# A simple OSC threading server to listen for OSC messages
server = osc_server.ThreadingOSCUDPServer((serverIp, serverPort), dispatcher)
print(f'Starting server on {server.server_address[0]}, port {server.server_address[1]}.')

# run our server
server.serve_forever()

In [ ]:
# closing the OSC server
server.server_close()

## Activity

1. Test and explore the example on your local machine. Use "localhost" or "127.0.0.1" as your client and server IPs.
2. Test and play around with the example on a Wi-Fi or wired network in pairs.